# Hands-on 3: Desafios relacionados ao exemplo third.cc

## Objetivos:

 Este Hands-on descreve  o funcionamento do exemplo third.cc, incluindo a resolução de três desafios como meta principal:
 
 1. envio de três pacotes de STAs diferentes;
 2. uso do NetAnim para verificar o movimento dos usuários;
 3. uso do exemplo rate-adaptation-distance.cc.
 
 

## Cenário:

O exemplo acrescenta uma rede Wi-Fi à topologia de rede utilizada no exemplo 2, sendo composta por uma conexão ponto-a-ponto entre uma rede sem fio e uma rede CSMA/CD. O primeiro nó da rede ponto-a-ponto é considerado o ponto de acesso da rede sem fio. 

![title](img/topologia.png)

## Requisitos
1. Ter feito o Hands-on 2
2. Ter instalado:
   1. ns-3
   2. NetAnim
   3. Gnuplot


## Versão desse tutorial

* ns-3.29.

## Como funciona o exemplo third.cc ?

Esta seção fornece uma descrição passo a passo do exemplo third.cc, a fim de fornecer conhecimento para a resolução dos desafios.

No início, são adicionadas todas as bibliotecas necessárias para a execução do código.


![title](img/1.png)

Após a habilitação dos componentes de registro, são definidas algumas variáveis que poderão ser alteradas através de linha de comando, sem a necessidade de recompilação do código. A variável "**verbose**" dirá se as componentes de registro a nível de informação para aplicações *echo* estarão habilitadas ou não. A variável "**tracing**" indicará se o rastreio *pcap* estará habilitado. E as variáveis "**nCsma**" e "**nWifi**", definem a quantidade de nós extras (além dos nós conectados ponto a ponto) na rede de barramento e na rede Wi-Fi, respectivamente. 

![title](img/2.png)

Duas condições são adicionadas: 1) uma se refere à necessidade do número de nós em uma rede Wi-Fi ser limitada à 18 nós; 2) as mensagens de registro para aplicação do cliente e servidor são habilitadas a nível informacional se a variável **verbose** for *true*. Consulte o tutorial do ns-3 para saber quais são os outros níveis das mensagens de registro.

**TODO Jéssika: colocar o link da documentação do ns-3 sobre as mensagens de registro.**

![title](img/3.png)

Neste bloco, está contido exatamente o que já foi visto no hands-on 2: São criados 2 nós para a rede ponto-a-ponto e **nCsma** nós da rede Csma (default=3). O segundo nó da rede ponto a ponto também é adicionado no *container* dos nós CSMA. Em seguida, ocorre a instalação dos dispositivos nos nós através de **CsmaHelper** e **PointToPointHelper**.

![title](img/4.png)

A partir daqui começa a expansão da topologia de rede criada no hands-on 2. São criados **nWifi** (*default = 3) nós extras, guardados no *container* "**wifiStaNodes**" e o primeiro nó da rede p2p é adicionado no *container*  **WifiApNode** para ser o ponto de acesso. 

![title](img/5.png)

Nas linhas de código abaixo são definidas as configurações da camada PHY para dispositivos Wi-Fi. Existem duas opções de modelagem da camada física disponíveis no ns-3: a classe "**YansWifiPhy**" não oferece decomposição do sinal em frequência, não servindo para simulações com efeitos dependentes da frequência (perda de propagação dependente da frequência, simulações de interferências de diferentes tecnologias em um mesmo canal, entre outras).  Já a classe "**SpectrumWifiPhy**" foi criada com base na "**YansWifiPhy**", mas também com base no *framework Spectrum*, o que permite simulações de efeitos dependentes de frequência. Somente objetos **ns3::YansWifiPhy** podem ser anexados a um objeto **ns3::YansWifiChannel**. Com este, não é possivel modelar interferência por canal adjacente, mesmo que os canais estejam sobrepostos.  

**TODO Jéssika: Ver se concorda. Se sim, apagar o parágrafo correto.** (ver as chaves) 

{
Na primeira linha da imagem abaixo, é instanciado um objeto do tipo "**YansWifiChannelHelper**" (que é o helper utilizado *helper* do tipo do canal), ele receberá o objeto *default* do canal **YansWifiChannel**. De modo semelhante, na linha seguinte, é instanciado um objeto do tipo "**YansWifiPhyHelper**" que receberá os valores *defaut* de **YansWifiPhy**. Se o canal e a camada física fossem criados diretamente através do instanciamento das classes **YansWifiChannel** e **YansWifiPhy**, seria necessário criar um conjunto de outros objetos e conectá-los apropriadamente. Com o *helper*, tudo é feito de forma automatica, sendo necessário apenas configurar os parâmetros e chamar o método *create.*


Na primeira linha da imagem abaixo é criado um *helper* através da classe "**YansWifiChannelHelper**", que ajudará a criar um canal **YansWifiChannel** *default*. Através da classe "**YansWifiPhyHelper**" é criado um *helper* que ajudará a criar uma camada física **YansWifiPhy** *default*. Se o canal e a camada física fossem criados diretamente através do instanciamento das classes **YansWifiChannel** e **YansWifiPhy**, seria necessário criar um conjunto de outros objetos e conectá-los apropriadamente. Com o *helper*, tudo é feito de forma automatica, sendo necessário apenas configurar os parâmetros e chamar o método *create.*
}

Os *defaults* dos *Helpers* podem ser alterados através dos seus métodos. EX: 

No canal,os *defaults* de  *PropagationLossModel* e *PropagationDelayModel* podem ser alterados através dos métodos **YansWifiChannelHelper::SetPropagationDelay** e **YansWifiChannelHelper::SetPropagationDelay**.

**TODO Jéssika: os dois métodos acima são iguais.** 
 
Na camada física, o *default* de **errorRateModel** pode ser alterado através de **YansWifiPhyHelper::SetErrorRateModel**. Frequência, MCS (*Modulation Coding Scheme*), *ShortGuardInterval* (para 802.11n/ac/ax), *greedfieldPreamble* (para 802.11n), *channelWidth*, suporte a MIMO (número de antenas e número de *spacial streams* - para 802.11 n/ac/ax), *guardinterval* (para 802.11ax) também podem ser alterados através de métodos da classe "**YansWifiPhyHelper**". Consulte *ns-3-model-library* para mais informações.

**TODO Jéssika: link para a documentação.**
 
Finalmente, na terceira linha, um objeto **YansWifiChannel** é criado através do método *create* do *helper* **channel**. Note que ainda não é criado um objeto **YansWifiPhy**, apenas é informado ao *helper* **phy** a qual canal a camada física deverá se conectar. A camada física será criada posteriormente. 

**TODO Jéssika: o texto acima está um pouco confuso, ver se concorda: Finalmente, na terceira linha, o objeto *phy* utiliza o método *SetChannel* para setar o canal utilizado na camada física. O canal utilizado é criado por meio do objeto *channel* utilizando o método *Create*.**

![title](img/6.png)


Em seguida, é criado o objeto "**wifi**" do tipo **WifiHelper** que ajudará a criar um dispositivo de rede Wi-Fi. O objeto *wifi* tem como *default* o padrão Wi-Fi 802.11a e o algoritmo de adaptação **ns3::ArfWifiManager**. É possível tanto alterar o algoritmo de adaptação (por meio do método **setRemoteStationManager**), quanto o padrão utilizado (por meio do método **SetStandard**). O método **SetStandard** alinha os parâmetros Phy e Mac para os *defaults* do padrão definido, sobrescrevendo valores que já possam ter sido configurados.  Por exemplo, se é definido o padrão 802.11n, então o *default* de *channelWidh* será  20 MHz; se o padrão for 802.11ac, então o *default* de *channelWidh* passa a ser 80 MHz. Caso o usuário não queira utilizar os *defaults* (ex: 802.11ac também suporta 160 MHz) ele mesmo poderá sobrescrever após a instalação do dispositivo no nó, através de **Config::Set:**.

Na segunda linha, O algoritmo de adaptação é alterado do *default* para o *Aarf*.
![title](img/7.png)

Aqui, é criado um **WifiMacHelper** que ajudará a criar um modelo de camada MAC para as STAs e será reutilizado para configurar o AP. Através de **WifiMacHelper**, os parâmetros como tipo de camada MAC, suporte à QoS, suporte a HT (802.11n), VHT (802.11ac) ou HE (802.11ax) podem ser configurados. Por *default*, o *helper* configura uma arquitetura *ad-hoc* sem suporte à QoS, HE, VHT e HT. Dependendo do tipo de infraestrutura configurada, se tem suporte a QoS ou suporte à VHT, HT, HE, vários outros parâmetros podem ser configurados. Consulte *ns-3-model-library* para informações detalhadas. 

**TODO Jéssika: link para a documentação.**

     EX:
     A característica MSDU é desabilitada por *default* para todas as categorias QoS, e MPDU é habilitada apenas        para AC_VI (trafégo de vídeo) e AC_BE (*best-effort traffic*) com agragação máxima de 65535 bytes.Se é definido suporte à QoS e à HT ou VHT ou HE, então tanto MSDU quanto MPDU podem ser configurados para uma categoria de acesso específica. 
     
**TODO Jéssika: não entendi o motivo dessa explicação acima.**

Na segunda linha, o SSID (nome da rede Wi-Fi) é definido. Na terceira linha, os parâmetros do *helper* são configurados: definição do tipo de camada MAC (sta), o nome da rede, e *probing* está desativado.
![title](img/8.png)

A função do método *install* da classe **WifiHelper** é criar um conjunto de dispositivos de rede Wi-Fi e instalá-los no conjunto de nós passado como parâmetro. A camada física e MAC de cada dispositivo é definida de acordo com os *helpers*  passados como parâmetro. O método *install* chama *phy.create* e *mac.create*, e os retornos passam a ser, respectivamente, a camada física e  mac de cada dispositivo criado.

**TODO Jéssika: sobre o phy.create e mac.create, é necessário colocar isso? Pode ser informação demais...**

![title](img/9.png)

Na imagem abaixo, as configurações do helper "**mac**" são alteradas para reutilização no ponto de acesso (AP). O AP recebe as mesmas configurações da camada PHY das *Stas* e na camada MAC, a configuração é alterada para que ele se comporte como um AP. 

![title](img/10.png)

O dispositivo de rede Wi-Fi é criado e instalado no nó AP.
![title](img/11.png)

Aqui é definido o modelo de mobilidade para a rede Wi-Fi, considerando que os nós não estarão fixos, com         exceção do nó definido como ponto de acesso. Primeiramente é configurado um modelo aleatório, que é instalado nos nós extras da rede Wi-Fi.

![title](img/12.png)

Em seguida, é configurado um modelo com posição constante, que é instalado no AP. 

![title](img/13.png)

Através do **InternetStackHelper**, a pilha de protocolos de internet é instalada em todos os nós. Os endereços da rede são configurados da mesma forma que no hands-on 2, acrescentando a rede 10.1.3.0 para os **staDevices** e **apDevices**.

![title](img/14.png)

Através de um **UdpEchoServerHelper** uma aplicação *echo* para servidor é configurada para funcionar na porta 9, criada e instalada no nó mais a direita da rede em barramento, que passa a ser o servidor.


![title](img/15.png)

A aplicação *echo* é configurada para funcionar na porta 9 e se conectar com o servidor contido no nó mais a direta da rede em barramento. Após a configuração de alguns atributos, a aplicação é criada e instalada no segundo nó (da esquerda para a direita) da rede Wi-Fi.

![title](img/16.png)

Na primeira linha, o **Ipv4GlobalRoutingHelper** é utilizado para fazer o roteamento na rede Wi-Fi: 

![title](img/17.png)

Uma condição é adicionada. Se a variável **tracing** (discutida no começo desta seção) for verdadeira, então o rastreamento *pcap* para as 3 redes é ativado. Para a rede ponto-a-ponto, é ativado em todos os nós. Para a rede Wi-Fi é ativado apenas no AP. E para a rede em barramento é ativado apenas para o último nó em modo promíscuo. 

![title](img/18.png)

## Resultados do exemplo third.cc

![title](img/resultado1.png)

## Desafios

Para realizar a etapa de desafios faça uma cópia do exemplo third.cc (~/ns-allinone-3.29/ns-3.29/examples/tutorial) e coloque na pasta ~/ns-allinone-3.29/ns-3.29/scratch. Em seguida, faça a compilação e siga o passo-a-passo:

## Desafio 01:

Enviar 3 pacotes de STAs diferentes. O primeiro pacote deve ser enviado pela última STA, o segundo pela penúltima e o terceiro pela antepenúltima. Note que o usuário pode alterar o número de STAs da rede através de linha de comando. O código deve funcionar para todos os casos. Este desafio foi divido em 4 partes:

### 1.1) 3 Pacotes Iguais de STA's Diferentes:

Como os pacotes são iguais, é necessário criar apenas uma aplicação, porém necessário criar 3 **ApplicationContainer** diferentes (1 para cada nó). Utilize o método "install" do "UdpEchoClientHelper" para instalar a aplicação nos 3 nós. Não instale nos 3 primeiros nós, e sim nos últimos 3 nós. 

Resposta: [aqui](img/parte1/pacotesde3stas/1.png)

Obs: como as aplicações podem ser simuladas no mesmo período de tempo, outra opção seria utilizar o método .add para que não seja necessário criar 3 containers de aplicação. Outra opção é adicionar os 3 últimos nós a um *container* de nós específico e utilizar apenas um *container* de aplicações, chamando o método *install* apenas uma vez.

#### Resultado:
Rode seu código. Sua saída deve ser semelhante a:

![title](img/parte1/pacotesde3stas/resultado2.png)


### 1.2 Os pacotes são diferentes 
Agora os 3 pacotes enviados devem ser diferentes. O tamanho do segundo deve ser o dobro do primeiro, e o terceiro deve ser o dobre do segundo. O primeiro pacote é enviado em 5s e o intervalo de envio entre os 3 pacotes deve ser de 5s. Além disso, configure as aplicações para encerrarem em 25s. 

#### Passo 1:

Se as aplicações irão encerrar em 25 segundos, o servidor também deve encerrar em 25 segundos. Nenhuma outra alteração no servidor é necessária por enquanto. 

Resposta: [aqui](img/parte1/pacotesdiferentes3stas/1.png)

#### Passo 2: 
Crie 3 aplicações diferentes, modificando os tamanhos do pacotes. Instale-as nos seus respectivos nós, alterando os tempos de início e encerramento de cada aplicação. 

Resposta: [aqui](img/parte1/pacotesdiferentes3stas/2.png)

#### Passo 3:
Rode seu código e confira os resultados, sua saída deve ser semelhante a:

![title](img/parte1/pacotesdiferentes3stas/resultado12.png)

### 1.3 Envio de vários pacotes

Agora cada STA deve enviar vários pacotes. A última envia 3 pacotes, com intervalos de 0.4 s. A penúltima envia 4 pacotes com intervalos de 0.8 s. A última envia 5 pacotes com intervalos de 1.2 s. Para isso, basta alterar as configurações no memento de criação das aplicações. 

Resposta: [aqui](img/parte1/3stasvariospacotes/1.png)

Sua saída deve ser semelhante a: 

![title](img/parte1/3stasvariospacotes/resultado13.png)


### 1.4 Alteração no Servidor: 2 Aplicações Instaladas.

Agora o servidor está localizado no penúltimo nó da rede em barramento. Além disso, ele possui 2 aplicações instaladas: a primeira continua funcionando através da porta 9 e a segunda funciona através da porta 7. As 2 últimas STAs se comunicam com a  primeira aplicação (porta 9) e a outra com a segunda aplicação (porta 7).

#### Passo 1:
Mantenha a primeira aplicação na porta 9, e crie outra na porta 7, utilizando o **UdpEchoServerHelper**. 

Resposta: [aqui](img/parte1/2apservidor/1.png)

#### Passo 2: 
As 2 aplicações começam e encerram ao mesmo tempo. Instale as 2 no penúltimo nó da rede em barramento, sem criar 2 objetos, através do método **.add**. 

Resposta: [aqui](img/parte1/2apservidor/2.png)

#### Passo 3: 
Mantenha as aplicações dos clientes da porta 9, alterando apenas a aplicação a ser instalada no antepenúltimo nó da rede.

Resposta: [aqui](img/parte1/2apservidor/3.png)

#### Resultado:

Rode seu código e confira os resultados: 

![title](img/parte1/2apservidor/resultado14.png)





## Desafio 02: Uso do NetANim Para Verificar Movimento Dos Usuários.

O NetANim é um um software independente que utiliza arquivos de rastreamento gerados durante simulações do ns-3 para exibir a topologia de rede e criar uma animação do fluxo de pacotes entre os nós (móveis ou estacionários). 

A classe **ns3::AnimationInterface** é a responsável pela criação do arquivo de rastreamento no formato *.xml*. A classe registra o fluxo de pacote apenas se houver o evento de recepção de pacote. Sendo assim, cada evento de transmissão deve estar ligado a um evento de recepção.

Com o *NetAnim*, além de vizualizar o fluxo de pacotes, é possível visualizar e exportar tabelas, com possibilidade de uso de vários filtros. Para mais informações sobre o software consulte o manual do ns-3.

#### Passo 1: Instale o NetAnim

Se o NetAnim ainda não estiver intalado, dentro da sua pasta do ns-3, execute os seguintes comandos no terminal:

$ hg clone http://code.nsnam.org/netanim

$ cd netanim 

$ make clean 

$ qmake NetAnim.pro 

$ make

$./NetAnim

#### Passo 2: 
Altere o código gerado na parte 1 do desafio 1. Primeiramente, a classe "netanim-module.h" deve ser incluída.

![title](img/parte2/1.png)


Antes de "Simulator::Run()", adicione a linha responsável pela geração do arquivo *.xml*. O objeto **anim** irá rastrear o fluxo de pacotes e exportar os dados para o arquivo. 

![title](img/parte2/2.png)

#### Passo 3: 
Para que a animação funcione corretamente, é necessário que todos os nós possuam modelo de mobilidade instalados. Instale um modelo de mobilidade em todos os nós estacionários. Uma opção é usar o mesmo modelo criado para ser instalado no ponto de acesso:

![title](img/parte2/32.png)

#### Passo 4: 
Rode seu código. Note que será criado um arquivo "nome_que_voce_escolheu.xml" na sua pasta do ns-3, seu arquivo deverá ser parecido com [este](Gráficos/third.xml). Abra o NetAnim digitando "./NetAnim" usando terminal no diretório do "NetAnim". Abra o arquivo gerado através da sua simulação (Este link mostra o funcionamento do NetAnim: https://www.youtube.com/watch?v=tz_hUuNwFDs).

Explore as inúmeras opções oferecidas pelo software. Ativando a opção "IP" é possível visualizar o número IP de cada nó. Para visualizar o fluxo de pacotes e movimentos dos nós, clique na seta verde. Você também pode vizualizar o fluxo passo-a-passo, utilizando uma das opções na barra de ferramentas ao lado esquerdo (explore). 

Na aba *Packets* é possível visualizar tabelas com o fluxo de pacotes, informando transmissor, receptor e tempo. É possível filtrar para que a tabela contenha apenas certos tipos de pacotes, ou pacotes relacionados à apenas um conjunto específico de nós. Note que sem nenhum filtro a tabela possui 48 pacotes.  


![title](img/parte2/34.png)


#### Passo 5:

Às vezes, em uma simulação, se deseja visualizar o fluxo entre os nós apenas em um intervalo de tempo específico. Por exemplo, na rede criada, em boa parte do tempo de simulação são criados apenas pacotes de beacon. É possível limitar o tempo no qual os pacotes serão rastreados. Além de visualizar apenas dados de maior utilidade, também existe a vantagem da geração de tabelas com menor conteúdo de dados. Acrescente a seguinte alteração:


![title](img/parte2/4.png)


Realize os mesmos passos para visualização do NetAnim. Na aba "Packets", você deve observar que agora a tabela possui apenas 12 pacotes. 

#### Passo 6:

Outro método muito importante é o "EnablePacketMetadata". Através dele é possível habilitar que as informações relacionadas ao tipo de pacote também sejam enviados para o arquivo .xml e visualizados no NetAnim. Acrescente a alteração e observe se seu resultado é semelhante a: 

![title](img/parte2/33.png)

Ainda existem outros métodos, como o "*UpdateNodeDescription*" e "*UpdateNodeSize*", que podem facilitar a visualização da topologia da rede. Para mais informações sobre outros métodos da classe
 "*AnimationIterface*", consulte o Doxygen do ns-3.
 
**TODO Jéssika: link para a documentação.**

## Desafio 3: Uso do exemplo "rate-adaptation-distance.cc":

### Introdução

Em redes Wi-Fi não se tenta transmitir sempre a uma taxa de transmissão constante. O uso de algoritmos que adaptam a taxa de transmissão de acordo com as variações do canal é essencial para que se consiga a melhor transmissão possível. Mudanças no trafégo por usuário, no número de estações conectadas à rede ou distância entre receptor e transmissor são exemplos de características que influenciam diretamente no valor da melhor taxa de transmissão. Existem vários tipos de algoritmos, os implementados no ns-3 são: ArfWifiManager, onoeWifiManager, ConstantRateWifiManager, MinstrelWifiManager, IdealWifiManager, AarfWifiManager, AmrrWifiManager, CaraWifiManager, RraaWifiManager, AarfcdWifiManager, ParfWifiManager, AparfWifiManager.

#### Aarf (Adaptive Automatic Rate Fallback)

É um mecanismo simples, que considera apenas as perdas por desvanescimento do canal. Ele diminui a taxa de transmissão para a próxima taxa mais baixa disponível após 2 pacotes sucessivos perdidos. Após 10 transmissões de sucesso, ele aumenta a taxa para a próxima mais elevada disponível. o Décimo primeiro pacote é um pacote de sondagem, caso ele seja perdido, a taxa volta para a anterior. Em canais instáveis, esse algoritmo causa flutuações na rede, pois elevar a taxa pode levar a perdas e consequentes elevações seguidas de diminuição da taxa, resultando em uma instabilidade no valor da taxa de transmissão. 

#### Onoe 

Os primeiros pacotes são transmitidos a uma taxa inicial; após 1 segundo de transmissão as informações sob as transmissões são analisadas. Quando menos de 10% dos pacotes não precisaram ser reetransmitidos, o crédito é aumentado. Quando o crédito excede 10, a taxa de transmissão é aumentada e o crédito é zerado. Já quando a reetransmissão acontece para mais de 10% dos pacotes, o crédito é diminuido. Quando o cŕedito tem valor abaixo de zero, a taxa é reduzida e o crédito é zerado.

Esse algorítmo se caracteriza por ser um pouco mais completo, porém é lento, devido as análises a cada 1 segundo. 

#### Minstrel

É um algoritmo mais avançado, que faz atualizações periódicas a cada 100 ms. 10% dos pacotes transmitidos são pacotes de sondagem enviados a taxas aleatórias com o intuito de obter informações sob o canal. Se um pacote é transmitido corretamente, não é necessário testar outras  taxas. Se a transmissão falha, o pacote é retransmitido utilizando outras taxas. Se o pacote é de sondagem, a sequência de tentativas é: taxa aleatória, taxa de melhor vazão útil, melhor probabilidade, taxa básica. Se o pacote é de carga útil, a sequência é: taxa de melhor vazão, próxima taxa de melhor vazão, melhor probabilidade, taxa básica. 

A fim de que em um canal com bom comportamento não exista perda de tempo com taxas baixas, a taxa aleatória é colocada em segundo lugar quando ela é menor que a melhor vazão útil. Assim o algoritmo sempre inicia com a maior taxa. Quando o canal não está bom, a transmissão irá falhar, e a taxa aleatória é colocada em tentativa. 

#### RRAA (Robust Rate Adaptation Algorithm)

Este algoritmo evita perdas de pacotes devido a colisão com um mecanismo de RTS adaptativo e é baseado na medição da perda de pacote em uma janela de transmissão. O algorimto inicia com a maior taxa de transmissão possível e cada taxa selecionada é utilizada durante toda a janela de transmissão. Ao fim da janela de transmissão, a taxa de perda de pacote é calculada e a taxa é atualizada na proxima janela. A taxa é reduzida para a próxima mais baixa quando a PLR é menor que a Pmtl (Limite Máximo de Perda Tolerável). A taxa é aumentada para a próxima superior quando a PLR é maior  que a *pori* (Limite Oportunístico de Aumento de Taxa). Se a PLR estiver entre os dois limites (*pori* e pmtl), a taxa é mantida, pois o canal nem está ruim para a taxa atual nem está preparado para um taxa maior. 

#### CARA (Strategy Collision-Aware Rate Adaptation)

Seu funcionamento é parecido com o aarf, porém ele usa RTS/CTS para reduizir colisões de pacotes.

**TODO Jéssika: interessante colocar a fonte das informações acima.**

### Como funciona o exemplo"rate-adaptation-distance.cc ?"

Este exemplo cria uma infraestrutura simples de dois nós, na qual um AP gera tráafico UDP para uma STA. A STA é configurada para se mover em relação ao AP, gerando como saída o plot de *throughput vs. distance*.

No início, são adicionadas todas as bibliotecas necessárias para a execução do código.



![title](img/rate_adaptation/1.png)






Após habilitação dos  componentes de registro é criada a classe **NodeStatistics** na qual são criados vários atributos públicos e privados. A definição de cada um dos atributos públicos é feita após a criação da classe e explicado posteriormente.

![title](img/rate_adaptation/2.png)

Na imagem acima, a variável **m_bytesTotal** irá representar a quantidade total de bytes transmitidos a cada intervalo de tempo de análise. Neste código, os nós se deslocam *1 m* a cada *1 s*, logo o intervalo de tempo "steptime" é 1 segundo, e "stepsize" é 1 metro. **m_bytesTotal** poderá ser modificada através de um dos atributos públicos definidos a seguir.

A variável  **m_output** guarda os dados de posição *versus* throughput para plotagem do gráfico através do gnuplot. Ela poderá ser alterada através de um dos atributos públicos definidos a seguir.


#### Definição de cada atributo público:

Primeiro, é definido o construtor da classe, o qual inicializa a variável privada **m_bytesTotal** contendo o valor zero sempre que o objeto da classe é instanciado.  


![title](img/rate_adaptation/3.png)
A função **RxCallback** pega o valor do tamanho do pacote passado como parâmetro, e adiciona ao valor já contido em **m_bytestotal**.

![title](img/rate_adaptation/4.png)

A função **SetPosition** muda a posição do nó no modelo de mobilidade de acordo com a variavél **position** recebida como parâmetro.
![title](img/rate_adaptation/5.png)

A função **GetPosition** retorna a posição atual do nó.

![title](img/rate_adaptation/6.png)

O objetivo da função **AdvancePosition** é colocar os dados de posição e taxa de transmissão atual do nó na variável **m_output** e, em seguida, mandar o nó para sua próxima posição.

Primeiro ele usa a função já definida **GetPosition** para pegar a posição atual do nó. Na segunda linha é calculado o valor de taxa de transmissão em Mbps através do valor total de bytes transmitidos e o valor do intervalo de tempo correspondente. A quarta linha adiciona os valores encontrados de posição e a taxa em **m_output**, que será utilizada para plotar o gráfico com todas as posições *versus* taxa.

A quinta linha define qual será a próxima posição do nó (o nó está se deslocando apenas no eixo x, no valor de **StepSize**, que nesse caso será definido como 1 m). Em seguida a função **SetPosition** é utilizada para mandar o nó para essa nova posição definida. Finalmente, a última linha garante que a função **AdvancePosition** será chamada a cada valor de **StepTime**, calculando o valor de posição e taxa em todos os intervalos de tempo.

![title](img/rate_adaptation/7.png)

Abaixo, se o logging está habilitado as mudanças no valor da taxa de transmissão poderão ser impressas. 

![title](img/rate_adaptation/8.png)

Finalmente, a função **GetDataFile()** é definida, a qual retorna a variável **m_output**. 

![title](img/rate_adaptation/9.png)




### Começando o código
O script começa definindo o *default* das suas principais variáveis, como qual padrão de rede Wi-Fi será utilizado, quais algoritmos de adaptação serão utilizados no AP e no STA, as posições iniciais dos nós e como eles se deslocam. Observe todas as variáveis. Em seguida é adicionado o código para que todas essas variáveis possam ser definidas através de linha de comando. Por último (na imagem abaixo), com o número de passos definido e o tempo de cada passo, é calculado o tempo total da simulação.

###### Sobre "ShortGuardInterval"
Uma das variáveis é **ShortGuardInterval**. *Short Guard Interval* é uma melhoria técnica que veio com o padrão de Wi-Fi 802.11 n. *Guard Interval* é o intervalo entre símbolos utilizado para evitar interferência intersimbólica. Normalmente o valor desse intervalo de guarda é 800 ns, com o "ShortGuardInterval" este intervalo passa a ser 400 ns. Com o *ShortGuardInterval*, há a diminuição de *overhead* e consequente aumento do *throughput*. Entretanto, em certas condições de canal, o *throughput* pode piorar quando este intervalo pequeno não é suficiente para evitar interferência intersimbólica. Esta opção só pode ser usada para os padrões 802.11 n/ac/ax. 

###### "Sobre RtsThreshold"
Uma das variáveis é o **RtsThreshold**. RTS/CTS podem ser utilizados em redes Wi-Fi para evitar colisões. Antes de enviar um pacote, uma sta deve enviar um RTS para o AP. **RtsThreshold** diz o tamanho máximo de bytes que uma STA pode enviar sem precisar enviar um novo RTS.

##### "Sobre BE_MaxampduSize"

MPDU é um dos tipos de agregação de quadros definidos a partir do 802.11 n. Transmitir um quadro em uma rede Wi-Fi possui *overhead* significativo. A agregação permite agregar quadros em um só, de forma a dimiuir o *overhead* total na rede. **BE_MaxampduSize** diz o tamanho máximo dessa agregação em bytes.

![title](img/rate_adaptation/10.png)




Na figura abaixo os nós são criados, a camada física é configurada e, por último, os *containers* de dispositivos de rede são criados. A criação e instalação dos dispostivos nos nós ocorrerá apenas posteriormente. 

Observe que agora, além de configurar o canal da camada física, através do comando "WifiPhy.Set()" foi configurado se será utilizado ou não o *Short Guard Interval* de acordo com o valor *booleano* da variável **ShortGuardInterval**. Muitas outras variáveis podem ser alteradas através do comando **WifiPhy.Set()**. Consulte o Doxygen do ns-3 para mais informações (Modules/wifi Models/YansWifiPhy).

**TODO Jéssika: link para a documentação.**

![title](img/rate_adaptation/11.png)


O próximo passo é a configuração da camada MAC e criação e instalação dos dispositivos. Mas essa configuração será diferente para cada padrão. Aqui, após a criação de um objeto **WifiHelper** uma sequência de condições *if's* define o que acontece em cada padrão:

Para 802.11a/b/c: após a criação de um **WifiMacHelper**, o algoritmo de adaptação utilizado pelo objeto **WifiHelper** é definido com o valor de **rtsThreshold**. A configuração da camada MAC e instalação do dispositivo na STA é realizada como já aprendido anteriormente. No AP, tudo ocorre de forma semelhante:
 
![title](img/rate_adaptation/12.png)
 
Para o 802.11 n: o mesmo ocorre, a diferença é que o valor **BE_MaxampduSize** também faz parte da configuração da camada MAC.
 
![title](img/rate_adaptation/13.png)    

Exatamente o mesmo procedimento ocorre para o 802.11 ac. Agora que todas as opções foram definidas, o dispositivo sta e Ap são adicionados aos *containers* de dispositivos Wi-Fi.

 ![title](img/rate_adaptation/14.png)
 



A linha abaixo sobrescreve o valor de *default* de *ChannelWidth* do padrão especificado para o valor contido em **ChWidth**.

 ![title](img/rate_adaptation/16.png)
 
 Em seguida um modelo de mobilidade constante é definido tanto para STA como para o AP. Apesar do modelo de mobilidade constante, fazemos a STA se mover em relação ao AP através de método **AdvancePosition** da classe **NodeStatistics**
 
 ![title](img/rate_adaptation/17.png)
 
 Um objeto da classe **NodeStatistics**, chamado **atpCounter** é criado, e **m_totalbytes** é inicializado com zero. Em seguida, o primeiro evento é agendado para ser executado durante (0.5 + *stepsTime*) segundos. O evento agendado é o método **AdvancePosition**, cujo retorno é guardado em **atpCounter** e seus argumentos são o nó STA, **stepsSize** e **stepsTime**. 
  ![title](img/rate_adaptation/18.png)
  
 
  
 



O pŕoximo passo é instalar a pilha de protocolos de internet e definir os endereços dos dispositivos: 
    
   ![title](img/rate_adaptation/20.png)
    
    

Abaixo, **PacketSinkHelper** ajuda a instalar um **PacketSinkApplication** em um conjunto de nós. Esta aplicação foi escrita para complementar a aplicação onoff, e se trata de uma aplicação simples apta a receber pacotes. Os parâmetros do *helper* são o endereço e porta do socket Rx e o protocolo de transporte a ser utilizado. A aplicação **PacketSink** possui uma fonte que rastreia quando um pacote é recebido e fornece acesso ao pacote recebido e endereço. Esta fonte será útil no código.

**OnOffHelper** ajuda a criar e instalar um **OnOffApplication** em um conjunto de nós. Os parâmetros são o protocolo usado para enviar o tráfego e o endereço para qual se deseja enviar. Uma aplicação *OnOff* simplesmente gera um tráfego para um único destino de acordo com um padrão *On/Off*: no estado *On* ele envia tráfego e no estado Off não. Os estados On e Off se alternam. o tráfego gerado no estado On é caracterizado pela taxa de dados e tamanho do pacote. No código abaixo, com o método **SetConstantRate** a taxa de dados é configurada para 420 mb/s e o tamanho do pacote é configurado para 1420 bytes. A aplicação irá começar em 0.5 segundos e finalizar de acordo com o valor de **simuTime**. A aplicação é instalada no AP, em seguida a aplicação **PacketSink** também é configurada para começar e finalizar nos mesmos tempos.

![title](img/rate_adaptation/21.png)





Na imagem abaixo, o primeiro **Config::Connect** conecta a fonte de rastreamento ao método **RxCallback** do objeto **atpCounter** da classe **NodeStatistics**. Sempre que um pacote é recebido o método **RxCallback** é chamado e portanto o valor de **m_totalbytes** é atualizado, e com o método **advancePosition** (chamado a cada **stepsTime** durante a simulação) o valor de *throughput* é calculado.

O segundo **Config::Connect** conecta a fonte de rastreamento **RateChange** ao método **RateCallback**, o qual irá imprimir cada mudança na velocidade se o **logging** estiver habilitado. Finalmente a simulação é iniciada, e o simulador é configurado  para finalizar em **simuTime** segundos.

![title](img/rate_adaptation/22.png)

Com a simulação finalizada, o arquivo de saida é criado, através de um objeto gnuplot todas as configurações do gráfico são definidas, e os dados de *throughput* *versus* distância são escritos no arquivo de saida através do método **GetDatafile**.

### Desafio 3
Gere o gráfico *throughput versus Distância* para uma STA se movendo a cada 2 metros a cada 2 segundos. Devem existir 100 pontos no gráfico. Altere o padrão para 802.11n, o algoritmo de adaptação para HTminstrel, e habilite o *ShortGuardInterval*.

Resposta: [aqui](CODES/rate-adaptation-distance.cc)

Para gerar o gráfico rode o comando no terminal dentro da pasta principal do ns-3: 
                            
                            gnuplot "nome-do-seu-arquivo".plt
Sua saída deverá ser semelhante a:
![title](img/rate_adaptation/23.png)

Gráfico disponível: [aqui](Gráficos/throughput-MinstrelHt.eps)

Dados do gráfico *.plt* disponíveis: [aqui](Gráficos/throughput-minstrelHT.plt)

